# RAG法律助手
**本节课学习目标:**
1. 什么是RAG？
2. 为什么要文档切片？
3. 表示学习与嵌入
4. 向量化过程分析
5. LangChain集成Ollama
6. RAG检索底层实现细节
7. 企业级RAG项目方案分析

## 1,什么是RAG？
   **RAG(Retrieval Augmented Generation)顾名思义，通过检索的方法来增强生成模型的能力**
   ![](./img/rag原理.jpg)

### 为什么需要RAG？
   **大模言模型(LLM)因为预训练的过程,加之本身是基于预测去生成内容的系统,导致它必然会有如下缺陷**  
     **- 知识时效性**  
     **- 推理局限性**  
     **- 专业领域盲区**  
     **- 幻觉问题**  


## 2,RAG项目实战
### 搭建过程
  #### 一，语料库准备
  
  #### 二，加载文档并且按需求切片
    首先要对文档进行数据清洗，去除不必要的数据(噪声数据)


In [ ]:
#导入项目环境
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap,RunnablePassthrough
from langchain.prompts import PromptTemplate
from datasets import Dataset
from ragas import evaluate
from dotenv import load_dotenv
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

# 1. 加载和分割文档
loader = TextLoader("./documents/中华人民共和国刑法.txt")
# 
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)
splits = text_splitter.split_documents(documents)
text_list = []
for i in splits:
    print(i)
    text_list.append(i)

prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""请根据检索的上下文回答问题。
    
    上下文：
    {context}

    问题：
    {query}

    答案："""
)

#### 三，加载本地Embedding模型
  ##### 加载本地模型后，将文档切片传入模型进行向量化

In [ ]:
# 2. 创建向量存储(使用本地嵌入模型)
embeddings = HuggingFaceEmbeddings(model_name="D:\\AIProjects\\modelscope\\BAAI\\bge-base-zh-v1___5")
vectorstore = Chroma.from_documents(
                                    #文档切片
                                    splits, 
                                    #本地Embedding模型
                                    embeddings,
                                    #通过Chroma将向量存储到本地
                                    persist_directory="./chrome_db"
                                    )


#### 四，通过Ollama集成本地LLM大模型并提问  
  

  
  
  

In [ ]:
# 3. 构建检索链-通过Ollama集成本地千问大语言模型
llm = Ollama(
    model="qwen2.5:1.5b",
    #温度参数
    temperature = 0.0,
    top_k = 5
    )


# 4. 构成LangChain问答链
qa_chain = RunnableMap(
    {"context": vectorstore.as_retriever(), "query": RunnablePassthrough()},
) | prompt | llm | StrOutputParser()

# 5. 提出问题,LLM回答
result = qa_chain.invoke("张三抢劫银行获得十万元,将受到什么判罚？")
print(result)

#### 五，引入ragas评估回答指标

In [ ]:
load_dotenv()

def evaluate_rag(query, true_answer=None):
    """
    完整RAG流程+评估
    """
    retriever = vectorstore.as_retriever()
    contexts = [doc.page_content for doc in retriever.invoke(query)]

    qa_chain = RunnableMap(
        {"context": retriever, "query": RunnablePassthrough()},
    ) | prompt | llm | StrOutputParser()
    
    # 2. 生成阶段
    answer = qa_chain.invoke(query)
    
    true_answer = ''' 
        根据中华人民共和国刑法的相关规定，某某如果抢劫获得十万元，并且符合其他法定情节（如多次抢夺、数额巨大等），那么他可能会面临以下刑罚:
        处三年以上十年以下有期徒刑，并处罚金；有严重情节的，处十年以上有期徒刑、无期徒刑或者死刑，并处罚金或者没收财产.   
    '''
    # 3. 准备评估数据
    eval_data = {
        "question": [query],
        "answer": [answer],
        "contexts": [contexts],
        "ground_truth":[true_answer]
    }
    
    # 4. 运行评估
    dataset = Dataset.from_dict(eval_data)
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall if true_answer else None,
        context_precision
    ]
    
    result = evaluate(dataset, metrics=metrics,llm=llm,embeddings=embeddings)

    result.upload()
    return {
        "answer": answer,
        "contexts": contexts,
        "evaluation": result
    }

evaluate_rag("张三抢劫银行获得十万元,将受到什么判罚？")

#### 六，优化检索过程提升回答准确度

In [ ]:
# 综合优化示例：重排序 + 查询扩展
from langchain.retrievers import MultiQueryRetriever,ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

'''
    1. 查询扩展
        MultiQueryRetriever是一种自动化提示调整过程的工具,它通过使用大型语言模型(LLM)从不同的角度生成多个查询视角,
    并为每个查询检索相关文档,最终通过所有查询的结果的独特并集,得到一个更大、更有潜力的相关文档集合。
'''
retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 10}),
    llm=llm
)

''' 
    2. 检索后压缩/重排序
        LLMChainExtractor是一个从大语言模型中提取信息的工具,它可以将LLM的输出进行压缩,去除冗余信息,保留关键内容
        from_llm(llm)是LLMChainExtractor的一个类方法,用于创建一个LLMChainExtractor实例,
    并将传入的语言模型与该实例关联起来。这样,后续就可以使用这个实例对文本进行压缩处理
        ContextualCompressionRetriever是一种上下文压缩检索器,它结合了基础检索器和文档压缩器,
    能够通过上下文压缩技术，将检索结果优化为用户真正需要的内容，避免浪费资源
        压缩的目的是去除检索结果中的冗余信息，减少信息量，同时保留关键内容，以便后续处理。
            例子1:
            ​原句​​:“我们应该重视教育公平,因为教育能够改变一个人的命运,而公平的教育机会是实现社会公正的重要途径。”
            ​​压缩后​​:“教育公平是实现社会公正的重要途径。”
            例子2:
            原句​​:“地球的自转产生了昼夜交替的现象,地球围绕太阳的公转产生了四季变化。”
​            ​压缩后​​:“地球自转产生昼夜交替,公转产生四季变化。”
        重排序是对检索到的文档进行重新评估和排序，以确保最相关且最有价值的检索结果能够优先被用作回答查询的上下文输入。

    压缩与重排序的结合
        ​基础检索​​:先使用基础的向量存储检索器进行初步查询,从文档库中找到与查询相关的文档。
    ​    压缩检索​​:使用LLMChainExtractor对检索到的文档进行压缩,去除与查询不相关的内容,减少信息量同时保留关键信息。
    ​    ​重排序​​:对压缩后的文档进行重新排序,使用重排序模型(如Reranker)对文档的相关性进行更精确的评估,筛选出最相关的文档,
    并按照相关性分数进行排序,确保最相关的文档优先被传递给LLM进行处理。
'''
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# 4. 构成LangChain问答链
qa_chain = RunnableMap(
    {"context": vectorstore.as_retriever(), "query": RunnablePassthrough()},
) | prompt | llm | StrOutputParser()

# 5. 提出问题,LLM回答
result = qa_chain.invoke("张三抢劫获得十万元,将受到什么判罚？")
print(result)